In [ ]:
!pip install fipy
#!pip install petsc4py
#!pip install pyamg
#!pip install pysparse
import numpy as np
from scipy import interpolate
from fipy import *

In [ ]:
# Defining problem
from fipy import *
from fipy.variables.faceGradVariable import _FaceGradVariable

##############################################################
# Problem parameters
#############################################################
D = 1.0
N = 20
#dL = D / N
dL = 1.0
viscosity = 1.1 # 0.0001
U = 0.1
pressureRelaxation = 1.0
velocityRelaxation = 1.0
sweeps = 200
Dp = 0.1


##############################################################
# Problem domain
#############################################################

#mesh = Grid2D(Lx=D*4, Ly=D, dx=dL, dy=dL)
#mesh = Grid2D(Lx=4*D, Ly=D, dx=1./16., dy=1./16.)
#mesh = Grid2D(Lx=4, Ly=4, nx=4, ny=4)
mesh = Grid2D(Lx=50, Ly=10, nx=50, ny=10)
#mesh = Grid2D(Lx=4., Ly=.2, nx=80, ny=4)
#mesh = Grid2D(Lx=10, Ly=1, nx=100, ny=10)


##############################################################
# Problem variables
#############################################################

pressure = CellVariable(mesh=mesh, name='pressure')
pressureCorrection = CellVariable(mesh=mesh)
xVelocity = CellVariable(mesh=mesh, name='X velocity', value=0.0)
yVelocity = CellVariable(mesh=mesh, name='Y velocity')
adv_xVelocity = CellVariable(mesh=mesh, name='X velocity', value=0.0)
adv_yVelocity = CellVariable(mesh=mesh, name='Y velocity')
porosity = CellVariable(mesh=mesh, value=0.0)
velocity_mag = (xVelocity**2 + yVelocity**2)**0.5

porosity[(mesh.x >= 0.4) & (mesh.x <= 0.6)] = 0.0 #0.5

epsilon = 1 - porosity
darcy = 150*viscosity/Dp**2*(1-epsilon)**2/epsilon**3
fh = 1.75/Dp*(1-epsilon)/epsilon**3*velocity_mag

velocity = FaceVariable(mesh=mesh, rank=1)
velocity[0] = 0.0 #U
velocity[1] = 0.0
phi      = FaceVariable(mesh=mesh, rank=1)


##############################################################
# Symbolic equations
#############################################################

xVelocityEq = -CentralDifferenceConvectionTerm(coeff=velocity) + \
               DiffusionTermCorrection(coeff=viscosity) - \
               pressure.grad.dot([1., 0.]) #-\
               #ImplicitSourceTerm(coeff=darcy) -\
               #ImplicitSourceTerm(coeff=fh)
yVelocityEq = -CentralDifferenceConvectionTerm(coeff=velocity) + \
               DiffusionTermCorrection(coeff=viscosity) - \
               pressure.grad.dot([0., 1.]) #-\
               #ImplicitSourceTerm(coeff=darcy) -\
               #ImplicitSourceTerm(coeff=fh)
               
apx = CellVariable(mesh=mesh, value=1.)
apy = CellVariable(mesh=mesh, value=1.)
ap_field = CellVariable(mesh=mesh, rank=1) #FaceVariable(mesh=mesh, rank=1)
#ap_field[0] = 1./apx * mesh.cellVolumes #1./ apx.arithmeticFaceValue*mesh._faceAreas * mesh._cellDistances
#ap_field[1] = 1./apy * mesh.cellVolumes #1./ apx.arithmeticFaceValue*mesh._faceAreas * mesh._cellDistances
pressureCorrectionEq = DiffusionTermCorrection(coeff=ap_field) + phi.divergence


##############################################################
# Boundary Conditions
#############################################################

xVelocity.constrain(0., mesh.facesBottom| mesh.facesTop)
xVelocity.constrain(U, mesh.facesLeft)
xVelocity.faceGrad.constrain(0. * mesh.faceNormals, where = mesh.facesRight)
yVelocity.constrain(0., mesh.facesBottom| mesh.facesTop)
yVelocity.constrain(0., mesh.facesLeft)
yVelocity.faceGrad.constrain(0. * mesh.faceNormals, where = mesh.facesRight)
adv_xVelocity.constrain(0., mesh.facesBottom| mesh.facesTop)
adv_xVelocity.constrain(U, mesh.facesLeft)
adv_xVelocity.faceGrad.constrain(0. * mesh.faceNormals, where = mesh.facesRight)
adv_yVelocity.constrain(0., mesh.facesBottom| mesh.facesTop)
adv_yVelocity.constrain(0., mesh.facesLeft)
adv_yVelocity.faceGrad.constrain(0. * mesh.faceNormals, where = mesh.facesRight)
#pressure.constrain(velocity[0]*velocity[0]/2, mesh.facesRight)
pressure.constrain(0., mesh.facesRight)
pressure.faceGrad.constrain(0. * mesh.faceNormals, where = mesh.facesLeft | mesh.facesBottom| mesh.facesTop)
#pressure.constrain(0., mesh.facesRight)
#xVelocity.faceGrad.constrain(0. * mesh.faceNormals, where = mesh.facesBottom| mesh.facesTop | mesh.facesRight)
# velocity.constrain(0, mesh.facesBottom| mesh.facesTop)
# velocity[0].constrain(U, mesh.facesLeft)
# velocity[1].constrain(0., mesh.facesLeft)

##############################################################
# Utils
#############################################################
volume = CellVariable(mesh=mesh, value=mesh.cellVolumes, name='Volume')
contrvolume = volume.arithmeticFaceValue

advection_velocity_x = xVelocity.value.copy()
advection_velocity_y = yVelocity.value.copy()

verbose = False
bool_face_variables = False
for sweep in range(1):

    ## solve the Stokes equations to get starred values
    xVelocityEq.cacheMatrix(); xVelocityEq.cacheRHSvector()
    xres = xVelocityEq.sweep(var=xVelocity, underRelaxation=velocityRelaxation)
    xmat = xVelocityEq.matrix
    xrhs = xVelocityEq.RHSvector

    yVelocityEq.cacheMatrix(); yVelocityEq.cacheRHSvector()
    yres = yVelocityEq.sweep(var=yVelocity, underRelaxation=velocityRelaxation)
    ymat = yVelocityEq.matrix
    yrhs = yVelocityEq.RHSvector

    if verbose:
      print ("------ Iteration:", sweep)
      print ("Momentum predictor x-matrix")
      print (xmat)
      print ("Momentum predictor x-rhs")
      print (xrhs)
      print ("ustar in x direction")
      print (xVelocity)
      print ("Momentum predictor y-matrix")
      print (ymat)
      print ("Momentum predictor y-rhs")
      print (yrhs)
      print ("vstar in y direction")
      print (yVelocity)

    ## update the ap coefficient from the matrix diagonal
    apx[:] = numerix.asarray(xmat.takeDiagonal())
    apy[:] = numerix.asarray(ymat.takeDiagonal())
    ap_field[0] = 1./apx #*mesh.cellVolumes
    ap_field[1] = 1./apy #*mesh.cellVolumes

    if verbose:
      print ("Ainv x-diection")
      print (ap_field[0])
      print ("Ainv y-diection")
      print (ap_field[1])

    ## update the coefficients of the H action
    H_x = xmat.copy(); H_y = ymat.copy();
    H_x.addAtDiagonal(-xmat.takeDiagonal()); H_y.addAtDiagonal(-ymat.takeDiagonal())
    H_x_f = numerix.dot(-H_x.matrix, xVelocity.value) + (xrhs - pressure.grad.dot([1., 0.]).value) #*mesh.cellVolumes
    H_y_f = numerix.dot(-H_y.matrix, yVelocity.value) + (yrhs - pressure.grad.dot([0., 1.]).value) #*mesh.cellVolumes

    if verbose:
      print ("Hhat x-diection")
      print (H_x_f)
      print ("Hhat y-diection")
      print (H_y_f)

    ## updating phis
    u_asterix = H_x_f/apx
    v_asterix = H_y_f/apy
    phi[0] = u_asterix.arithmeticFaceValue
    phi[1] = v_asterix.arithmeticFaceValue

    if verbose:
      print ("Ainv*Hhat x-diection")
      print (u_asterix)
      print ("Ainv*Hhat y-diection")
      print (v_asterix)

    ## solve the pressure correction equation
    # pressureCorrectionEq = DiffusionTermCorrection(coeff=ap_field) + phi.divergence
    pressureCorrectionEq.cacheMatrix(); pressureCorrectionEq.cacheRHSvector()
    pres = pressureCorrectionEq.sweep(var=pressure)
    pmat = pressureCorrectionEq.matrix
    prhs = pressureCorrectionEq.RHSvector

    if verbose:
      print ("Pressure predictor matrix")
      print (pmat)
      print ("Pressure predictor rhs")
      print (prhs)
      print ("pressure")
      print (pressure)

    # Updating velocity field
    if bool_face_variables:
      velocity[0] = pressureRelaxation*(phi[0] - (ap_field[0] * pressure.grad.dot([1., 0.])).arithmeticFaceValue) + (1-pressureRelaxation) * velocity[0]
      velocity[1] = pressureRelaxation*(phi[1] - (ap_field[1] * pressure.grad.dot([0., 1.])).arithmeticFaceValue) + (1-pressureRelaxation) * velocity[1]
    else:
      adv_xVelocity = pressureRelaxation*(u_asterix[:] - (ap_field[0] * pressure.grad.dot([1., 0.]))[:]) + (1-pressureRelaxation)*adv_xVelocity
      adv_yVelocity = pressureRelaxation*(v_asterix[:] - (ap_field[1] * pressure.grad.dot([0., 1.]))[:]) + (1-pressureRelaxation)*adv_yVelocity

    if verbose:
      print ("Corrected velocity x-direction")
      print (pressureRelaxation*(u_asterix - (ap_field[0] * pressure.grad.dot([1., 0.]))) + (1-pressureRelaxation)*xVelocity)
      print ("Corrected velocity y-direction")
      print (pressureRelaxation*(v_asterix - (ap_field[1] * pressure.grad.dot([0., 1.]))) + (1-pressureRelaxation)*yVelocity)

    if bool_face_variables:
      velocity[..., mesh.facesBottom.value | mesh.facesTop.value] = 0.
      velocity[0, mesh.facesLeft.value] = U
      velocity[1, mesh.facesLeft.value] = 0
    else:
      velocity[0] = adv_xVelocity.arithmeticFaceValue
      velocity[1] = adv_yVelocity.arithmeticFaceValue

    if sweep%10 == 0:
      print('sweep:', sweep, ', x residual:', xres, \
                            ', y residual', yres, \
                            ', p residual:', pres, \
                            ', continuity:', max(abs(prhs)))
    
      print('u_x: ', np.mean(xVelocity.value))
      print('u_x_res: ', xres)
      print('u_y: ', np.mean(yVelocity.value))
      print('u_y_res: ', yres)
      print('phi_0: ', np.mean(phi[0].value))
      print('phi_1: ', np.mean(phi[1].value))
      print('p: ', np.mean(pressure.value))
      print('p_x: ', np.mean(pressure.grad[0].value))
      print('p_y: ', np.mean(pressure.grad[1].value))
      print('vel_x: ', np.mean(velocity[0]))
      print('vel_y: ', np.mean(velocity[1]))



plot = True
if plot:
  import matplotlib.pyplot as plt

  xc, yc = mesh.x.value, mesh.y.value

  plt.figure(figsize=[8,2])
  plt.tricontourf(xc, yc, xVelocity.value, 30)
  plt.colorbar()
  plt.title('X-Velocity')

  plt.figure(figsize=[8,2])
  plt.tricontourf(xc, yc, yVelocity.value, 30)
  plt.colorbar()
  plt.title('V-Velocity')

  plt.figure(figsize=[8,2])
  plt.tricontourf(xc, yc, pressure.value, 30)
  plt.colorbar()
  plt.title('Pressure')

/usr/local/lib/python3.7/dist-packages/fipy/solvers/scipy/linearLUSolver.py:41: RuntimeWarning: invalid value encountered in double_scalars
  if (numerix.sqrt(numerix.sum(errorVector**2)) / error0)  <= self.tolerance:
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:88: UserWarning: getRHSvector should be called after cacheRHSvector() and either sweep() or solve()
                          to calculate the RHS vector. None will be returned on this occasion.


sweep: 0 , x residual: 0.894427190999916 , y residual 0.0 , p residual: 0.025495218608672196 , continuity: 0.004995054277479829
sweep: 10 , x residual: 0.007930731815256619 , y residual 0.0011270342484914558 , p residual: 6.622845772083419e-05 , continuity: 4.100232119312739e-05
sweep: 20 , x residual: 0.0009187461753467085 , y residual 0.0003203390833166812 , p residual: 8.066541736680597e-06 , continuity: 8.729124559093414e-06
sweep: 30 , x residual: 0.00021795398699824957 , y residual 0.00010047782883993713 , p residual: 1.841575131553598e-06 , continuity: 2.541201102738403e-06
sweep: 40 , x residual: 6.855840551608653e-05 , y residual 2.3098027239041665e-05 , p residual: 5.092775873657418e-07 , continuity: 7.794326348905906e-07
sweep: 50 , x residual: 1.974940219195787e-05 , y residual 5.202040124780389e-06 , p residual: 1.669156676908861e-07 , continuity: 4.768064696168871e-07
sweep: 60 , x residual: 8.716831958412044e-06 , y residual 3.5171264157172607e-06 , p residual: 7.8993656